## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Aklavik,CA,68.2191,-135.0107,-31.97,66,20,4.61,few clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,63.00,72,75,12.66,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,66.31,52,75,35.68,broken clouds
3,3,Kodiak,US,57.7900,-152.4072,35.11,80,0,3.44,clear sky
4,4,Nusaybin,TR,37.0778,41.2178,43.20,47,1,11.86,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
5,5,Rikitea,PF,-23.1203,-134.9692,78.58,73,41,13.00,scattered clouds
8,8,Iquitos,PE,-3.7481,-73.2472,82.44,83,75,9.22,broken clouds
13,13,Inirida,CO,3.8653,-67.9239,80.37,84,55,3.56,broken clouds
28,28,Hithadhoo,MV,-0.6000,73.0833,82.69,74,100,11.50,overcast clouds
36,36,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,78.60,89,40,2.30,scattered clouds
38,38,Santa Helena,BR,-2.2311,-45.3000,82.20,79,99,4.05,overcast clouds
45,45,Puerto Ayora,EC,-0.7393,-90.3518,84.16,89,53,4.00,broken clouds
47,47,Bathsheba,BB,13.2167,-59.5167,83.01,57,20,18.41,few clouds
48,48,Hermanus,ZA,-34.4187,19.2345,88.90,56,53,22.06,broken clouds
49,49,Conde,BR,-7.2597,-34.9075,87.26,62,75,12.66,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID            0
City               0
Country            0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID            203
City               203
Country            203
Lat                203
Lng                203
Max Temp           203
Humidity           203
Cloudiness         203
Wind Speed         203
Current Weather    203
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
5,Rikitea,PF,78.58,scattered clouds,-23.1203,-134.9692,
8,Iquitos,PE,82.44,broken clouds,-3.7481,-73.2472,
13,Inirida,CO,80.37,broken clouds,3.8653,-67.9239,
28,Hithadhoo,MV,82.69,overcast clouds,-0.6000,73.0833,
36,Sao Gabriel Da Cachoeira,BR,78.60,scattered clouds,-0.1303,-67.0892,
38,Santa Helena,BR,82.20,overcast clouds,-2.2311,-45.3000,
45,Puerto Ayora,EC,84.16,broken clouds,-0.7393,-90.3518,
47,Bathsheba,BB,83.01,few clouds,13.2167,-59.5167,
48,Hermanus,ZA,88.90,broken clouds,-34.4187,19.2345,
49,Conde,BR,87.26,broken clouds,-7.2597,-34.9075,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.count()

City               203
Country            203
Max Temp           203
Current Weather    203
Lat                203
Lng                203
Hotel Name         203
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
clean_hotel_df.loc[666]

City                    Lodja
Country                    CD
Max Temp                75.51
Current Weather    light rain
Lat                   -3.4833
Lng                   23.4333
Hotel Name                   
Name: 666, dtype: object

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))